## Kymograph Cluster Code

should I add a retry call by default?

In [ ]:
import trenchripper as tr

In [ ]:
!sbatch -p transfer -t 0-12:00 --wrap="rsync -r /n/files/SysBio/PAULSSON\ LAB/Daniel/Image_Data/Bacillus_Project/2019-05-13_bacillus_training_data/ /n/scratch2/de64/2019-05-13_bacillus_training_data"

#### Start Dask

In [ ]:
# kymo_controller = tr.cluster.dask_controller(walltime='04:00:00',local=False,n_workers=40,memory='2GB')
kymo_controller = tr.cluster.dask_controller(
    walltime="04:00:00", local=False, n_workers=40, memory="4GB"
)
kymo_controller.startdask()
kymo_controller.daskcluster.start_workers()

In [ ]:
kymo_controller.daskclient

#### Perform Extraction

In [ ]:
# hdf5_extractor = tr.ndextract.hdf5_fov_extractor("/n/scratch2/de64/full_pipeline_test/40x_Ph2_Test_1.5.nd2","/n/scratch2/de64/full_pipeline_test")
hdf5_extractor = tr.ndextract.hdf5_fov_extractor(
    "/n/scratch2/de64/2019-05-13_bacillus_training_data/NDExp_Seq0000.nd2",
    "/n/scratch2/de64/2019-05-13_bacillus_training_data",
)

In [ ]:
hdf5_extractor.fov_metadata[:5]

In [ ]:
print(hdf5_extractor.exp_metadata["fields_of_view"])

In [ ]:
fov_list = list(range(0, 40))

In [ ]:
hdf5_extractor.writemetadata()

In [ ]:
kymo_controller.mapfovs(hdf5_extractor.extract_fov, fov_list)

In [ ]:
kymo_controller.retry_failed()

In [ ]:
kymo_controller.retry_processing()

In [ ]:
# props = kymo_controller.daskclient.gather(kymo_controller.futures) #this will hang until all futures are done

In [ ]:
# hdf5_extractor.writemetadata()
hdf5_extractor.extract_fov(4)

In [ ]:
# test = tr.utils.pandas_hdf5_handler("/n/scratch2/de64/full_pipeline_test/metadata.hdf5")

#### End Dask, Cancel Jobs

In [ ]:
kymo_controller.shutdown()

In [ ]:
# scancel --name=dask-worker --user=de64

#### Generate Kymographs

#### Start Dask

In [ ]:
kymo_controller = tr.cluster.dask_controller(
    walltime="04:00:00", local=False, n_workers=40, memory="4GB"
)
kymo_controller.startdask()
kymo_controller.daskcluster.start_workers()

In [ ]:
kymo_controller.daskclient

In [ ]:
kymo_controller.daskclient.restart()

In [ ]:
# headpath = "/n/scratch2/de64/full_pipeline_test"
# all_channels = ["channel_RFP","channel_BF"]

# kymo = tr.kymograph.kychunker(headpath=headpath,all_channels=all_channels,trench_len_y=270,padding_y=40,trench_width_x=30,\
#                  t_chunk=25,t_range=(0,-1),y_percentile=90,y_min_edge_dist=80,smoothing_kernel_y=(17,1),triangle_nbins=50,triangle_scaling=1.60,\
#                  expected_num_rows=2,orientation_detection=0,x_percentile=95,background_kernel_x=(21,1),smoothing_kernel_x=(9,1),otsu_nbins=50,otsu_scaling=0.7)


headpath = "/n/scratch2/de64/2019-05-13_bacillus_training_data/"
all_channels = ["channel_RFP", "channel_Phase"]

kymo = tr.kymograph.kychunker(
    headpath=headpath,
    all_channels=all_channels,
    trench_len_y=200,
    padding_y=20,
    trench_width_x=20,
    t_chunk=100,
    t_range=(0, -1),
    y_percentile=60,
    y_min_edge_dist=80,
    smoothing_kernel_y=(17, 1),
    triangle_nbins=50,
    triangle_scaling=1.1,
    expected_num_rows=2,
    orientation_detection=0,
    orientation_on_fail=1,
    x_percentile=99,
    background_kernel_x=(21, 1),
    smoothing_kernel_x=(9, 1),
    otsu_nbins=50,
    otsu_scaling=0.25,
)

In [ ]:
# Y Percentile 60
# Y Smoothing Kernel 17
# Triangle Trheshold Bins 50
# Triangle Threshold Scaling 1.1
# Minimum Trench Length 50
# Y Padding 20
# Trench Length 200
# Orientation Detection Method 0
# Expected Number of Rows (Manual Orientation Detection) 2
# Top Orientation when Row Drifts Out (Manual Orientation Detection) 0
# X Percentile 99
# X Background Kernel 21
# X Smoothing Kernel 9
# Otsu Trheshold Bins 50
# Otsu Threshold Scaling 0.33
# Trench Width 26

In [ ]:
# fov_list = list(range(0,80))
fov_list = list(range(0, 40))

In [ ]:
kymo_controller.mapfovs(kymo.generate_kymograph, fov_list, retries=1)

In [ ]:
kymo_controller.retry_failed()

In [ ]:
kymo.collect_metadata(fov_list, use_archive=False, overwrite_archive=True)

In [ ]:
kymo.collect_metadata(fov_list, use_archive=True, overwrite_archive=False)

maybe an estimate of memory usage

In [ ]:
kymo.generate_kymograph(0)

In [ ]:
import h5py
from matplotlib import pyplot as plt

In [ ]:
# for fov in fov_list:
for fov in [12, 63]:
    print(fov)
    with h5py.File(
        "/n/scratch2/de64/full_pipeline_test/hdf5/fov_" + str(fov) + ".hdf5"
    ) as testfile:
        data = testfile["channel_RFP"]
        arr = data[:, :, 0]

In [ ]:
arr.shape

In [ ]:
import numpy as np

arr = np.array([1, 2, 3, 4])
arr[np.array([0, 2, 3])]

In [ ]:
import pandas as pd

In [ ]:
meta_handle = tr.utils.pandas_hdf5_handler(
    "/n/scratch2/de64/2019-05-10_bacillus_training_data/metadata.hdf5"
)
global_meta = meta_handle.read_df("global", read_metadata=True)

In [ ]:
global_meta.metadata

In [ ]:
kymo_handle = tr.utils.kymo_handle()
kymo_handle.import_wrap(h5py_handle["0/channel_RFP"][39])
plt.imshow(kymo_handle.return_unwrap())

#### Generate Segmentation

#### Start Dask

In [ ]:
kymo_controller = tr.cluster.dask_controller(
    walltime="01:00:00", local=False, n_workers=80, memory="1GB"
)
kymo_controller.startdask()
kymo_controller.daskcluster.start_workers()

In [ ]:
kymo_controller.daskclient

In [ ]:
kymo_controller.daskclient.restart()

In [ ]:
# segment = tr.segment.fluo_segmentation_cluster("/n/scratch2/de64/full_pipeline_test","channel_RFP",smooth_sigma=0.75,wrap_pad=3,hess_pad=4,min_obj_size=30,cell_mask_method='local',\
#                  cell_otsu_scaling=1.,local_otsu_r=15,edge_threshold_scaling=1.,threshold_range=20,threshold_step=10,convex_threshold=0.8)
segment = tr.segment.fluo_segmentation_cluster(
    "/n/scratch2/de64/2019-05-10_bacillus_training_data/",
    "channel_RFP",
    smooth_sigma=0.75,
    wrap_pad=3,
    hess_pad=4,
    min_obj_size=30,
    cell_mask_method="local",
    cell_otsu_scaling=1.0,
    local_otsu_r=15,
    edge_threshold_scaling=1.0,
    threshold_range=20,
    threshold_step=10,
    convex_threshold=0.8,
)

In [ ]:
# fov_list = list(set(range(0,80))-set([77]))
fov_list = list(set(range(0, 40)) - set([34, 35, 36, 37, 38, 39, 9, 10, 11, 17]))

In [ ]:
kymo_controller.mapfovs(segment.generate_segmentation, fov_list)

In [ ]:
segment.generate_segmentation(10)

In [ ]:
kymo_controller.daskclient.get_futures_error(kymo_controller.futures[1])

#### End Dask, Cancel Jobs

In [ ]:
kymo_controller.shutdown()

In [ ]:
import h5py
from matplotlib import pyplot as plt
import numpy as np
import skimage as sk

In [ ]:
data = h5py.File("/n/scratch2/de64/2019-05-10_bacillus_training_data/hdf5/fov_1.hdf5")

In [ ]:
array = data["channel_RFP"][:, :, 10]

In [ ]:
plt.hist(array.flatten(), bins=100)
plt.show()

In [ ]:
plt.imshow(array[470:500, 175:190])

In [ ]:
array[470:500, 175:190]

In [ ]:
np.percentile(data["channel_RFP"][:, :, 0], 90)

In [ ]:
kymo_handle = tr.utils.kymo_handle()

In [ ]:
list(data.keys())

In [ ]:
kymo_handle.import_wrap(data["19"])
plt.imshow(kymo_handle.return_unwrap())

### NOTE

- **How to deal with some fovs having a different number of rows than other fovs??? Try to make more flexible...**

maybe add a kymograph viewer here

### Note
There may be an issue where hdf5 files do not close properly on crashes...

In [ ]:
# props = kymo_controller.daskclient.gather(kymo_controller.futures) #this will hang until all futures are done

#### End Dask, Cancel Jobs

In [ ]:
kymo_controller.shutdown()

In [ ]:
# scancel --name=dask-worker --user=de64

#### Note: access request to the transfer partition pending; automate this bit later

In [ ]:
import subprocess

in_path = "/n/files/SysBio/PAULSSON LAB/Daniel/Image_Data/test"
out_path = "/n/scratch2/de64/for_sylvia/"
wrap_str = "rsync " + in_path + " " + out_path
cmd_str = 'sbatch -p transfer -t 0-12:00 --wrap="' + wrap_str + '"'

subprocess.check_output(cmd_str, shell=True)

In [ ]:
cmd_str

In [ ]:
!sbatch -p transfer -t 0-12:00 --wrap="rsync /n/files/SysBio/PAULSSON\LAB/Daniel/Image_Data/Bacillus_Project/test /n/scratch2/de64/for_sylvia/"

#### Note: need to find a way to associate lanes from init barcoding snapshot with imaging lanes

plan:
- extract and segment (with phase) barcoding file
- extract and segment experiemnt file
- for both, note x positions of t=0 trenches
- compare these values to map trenches to eachother...